In [ ]:
import torch
import os
import sys
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import random
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

import config
from model import get_model
from xai_utils import visualize_gradcam, preprocess_image_for_xai


In [ ]:
print(f"Loading best trained model from: {config.SAVE_MODEL_PATH}")
model = get_model(config.MODEL_NAME, config.NUM_CLASSES, pretrained=False)
model.load_state_dict(torch.load(config.SAVE_MODEL_PATH, map_location=config.DEVICE))
model.to(config.DEVICE)
model.eval()
print("Model loaded successfully.")

Loading best trained model from: d:\Job\Atharva\Projects\marine-species-classification\outputs\models\resnet50_best_model.pth


d:\Job\Atharva\Projects\marine-species-classification\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Job\Atharva\Projects\marine-species-classification\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model loaded successfully.


In [3]:
temp_dataset = datasets.ImageFolder(config.TRAIN_DIR)
class_names = temp_dataset.classes
class_to_idx = temp_dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}
print(f"Class names: {class_names}")

Class names: ['Fish', 'Goldfish', 'Harbor seal', 'Jellyfish', 'Lobster', 'Oyster', 'Sea turtle', 'Squid', 'Starfish']


In [ ]:
print("\nSelecting random images from the test set for XAI analysis...")
test_dataset_paths = []
for class_folder in os.listdir(config.TEST_DIR):
    class_path = os.path.join(config.TEST_DIR, class_folder)
    if os.path.isdir(class_path):
        for img_name in os.listdir(class_path):
            test_dataset_paths.append(os.path.join(class_path, img_name))

num_images_to_analyze = 5
selected_image_paths = random.sample(test_dataset_paths, num_images_to_analyze)



Selecting random images from the test set for XAI analysis...


In [ ]:
print("Enabling gradients for all model parameters...")
for param in model.parameters():
    param.requires_grad = True

print("Model structure:")
for name, module in model.named_modules():
    if 'layer4' in name:
        print(f"  {name}: {type(module)}")

print(f"\nFinal layer structure: {model.fc}")

print("\nGenerating Grad-CAM visualizations...")
for img_path in selected_image_paths:
    print(f"Processing: {img_path}")
    
    
    input_tensor, original_img = preprocess_image_for_xai(
        img_path, config.IMAGE_SIZE, config.IMAGENET_MEAN, config.IMAGENET_STD, config.DEVICE
    )

    
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = torch.softmax(output, dim=1)
        predicted_prob, predicted_idx = torch.max(probabilities, 1)
    
    predicted_class_name = idx_to_class[predicted_idx.item()]
    print(f"  Predicted class: {predicted_class_name} (Confidence: {predicted_prob.item():.4f})")

    
    visualize_gradcam(
        model,
        input_tensor,
        img_path,
        predicted_idx.item(),
        class_names,
        config.MODEL_NAME,
        config.VISUALIZATIONS_DIR
    )

print("\nGrad-CAM analysis complete. Check the 'outputs/visualizations' folder for images.")

Enabling gradients for all model parameters...
Model structure:
  layer4: <class 'torch.nn.modules.container.Sequential'>
  layer4.0: <class 'torchvision.models.resnet.Bottleneck'>
  layer4.0.conv1: <class 'torch.nn.modules.conv.Conv2d'>
  layer4.0.bn1: <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
  layer4.0.conv2: <class 'torch.nn.modules.conv.Conv2d'>
  layer4.0.bn2: <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
  layer4.0.conv3: <class 'torch.nn.modules.conv.Conv2d'>
  layer4.0.bn3: <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
  layer4.0.relu: <class 'torch.nn.modules.activation.ReLU'>
  layer4.0.downsample: <class 'torch.nn.modules.container.Sequential'>
  layer4.0.downsample.0: <class 'torch.nn.modules.conv.Conv2d'>
  layer4.0.downsample.1: <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
  layer4.1: <class 'torchvision.models.resnet.Bottleneck'>
  layer4.1.conv1: <class 'torch.nn.modules.conv.Conv2d'>
  layer4.1.bn1: <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
  l